<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=805fb46fa4cd89344990d66d5d6feb1c40996932eb7fd473c8ac21f3359f1859


  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 11:02:46
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: -1.47 L (-1.04%)
Current PnL: -21.20 L (-14.12%)
CY Booked + Current PnL: -9.76 L (-6.5%)
-------------------
Total profit:  1.50 L
Total loss:  -22.70 L
-------------------
Total Booked + Current PnL: 17.22 L (13.99%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.71 L (63.19%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,63.0,H-MC,14.51,118402.0,17898.0,11213.0,-0.40,17.81,9.47,28.96,79.0,1.60,0.85,57.82,MH,ATH,METALS
19,CIPLA,1662.74,-20.47,55.0,X-LC,3.74,212583.0,8083.0,15221.0,-0.65,3.95,7.16,11.39,10.0,0.53,1.52,13.26,X40N,ATH,PHARMA
77,TTKPRESTIG,770.00,93.81,41.0,M-SC,1.26,83493.0,-17284.0,17375.0,0.33,-17.15,20.81,0.09,245.0,-0.99,0.60,9.69,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.94,49.0,X-MC,3.31,189800.0,25130.0,18202.0,-2.62,15.26,9.59,26.31,89.0,1.38,1.36,24.46,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,50.0,X-MC,1.04,206535.0,14793.0,22967.0,-2.85,7.72,11.12,19.69,99.0,0.64,1.48,15.04,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,528.00,1992.23,37.0,H-SC,3.83,84430.0,-26602.0,50211.0,1.80,-23.96,59.47,21.26,156.0,-0.53,0.60,14.07,OX40N,NTT,MISC
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.52,127682.0,4346.0,127286.0,0.74,3.52,99.69,106.72,119.0,0.03,0.91,27.73,AR,ATH,MISC
5,ASIANPAINT,3465.66,-18.03,37.0,X-LC,5.48,206994.0,-44774.0,97991.0,0.51,-17.78,47.34,21.13,27.0,-0.46,1.48,11.34,X40,ATH,PAINTS
77,TTKPRESTIG,770.00,93.81,41.0,M-SC,1.26,83493.0,-17284.0,17375.0,0.33,-17.15,20.81,0.09,245.0,-0.99,0.60,9.69,OX40N,NTT,DURABLES
35,ICICIPRULI,790.00,-21.22,49.0,X-MC,2.02,135464.0,-324.0,43078.0,0.28,-0.24,31.80,31.48,107.0,-0.01,0.97,11.85,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-5.61,36.0,H-LC,1.24,151595.0,-34500.0,81983.0,-3.67,-18.54,54.08,25.51,15.0,-0.42,1.08,11.71,AR,ATH,ELECTRICAL
55,RAJOOENG,143.1,-41.05,45.0,H-SC,16.79,89440.0,-13060.0,53664.0,-3.46,-12.74,60.00,39.61,114.0,-0.24,0.64,9.23,AR,ATH,MISC
82,VALIANTORG,838.0,-287.77,28.0,H-SC,2.35,104295.0,-59310.0,180629.0,-3.08,-36.25,173.19,74.15,139.0,-0.33,0.75,33.69,XR,NTT,CHEMICALS
26,GREENPANEL,537.0,228.17,55.0,M-SC,4.49,146595.0,-36483.0,115458.0,-3.03,-19.93,78.76,43.14,230.0,-0.32,1.05,37.89,XY24,NTT,MISC
84,VOLTAS,1530.0,-1.50,50.0,X-MC,1.04,206535.0,14793.0,22967.0,-2.85,7.72,11.12,19.69,99.0,0.64,1.48,15.04,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,42.74,48.0,M-MC,12.59,225806.0,844.0,166803.0,-0.79,0.38,73.87,74.53,192.0,0.01,1.62,32.75,XY24,BTT,STEEL
86,WIPRO,420.0,-13.79,48.0,M-LC,5.84,151720.0,775.0,108677.0,-1.60,0.51,71.63,72.51,53.0,0.01,1.09,6.57,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,62.0,H-SC,11.31,131193.0,-11286.0,30804.0,-0.39,-7.92,23.48,13.70,163.0,-0.37,0.94,54.84,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,41.0,M-SC,1.26,83493.0,-17284.0,17375.0,0.33,-17.15,20.81,0.09,245.0,-0.99,0.60,9.69,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.97,220653.0,-49014.0,85349.0,-0.94,-18.18,38.68,13.47,138.0,-0.57,1.58,12.20,XY24,NTT,PAINTS
18,CERA,9475.0,-25.30,30.0,H-SC,1.94,136045.0,-39858.0,81885.0,-0.87,-22.66,60.19,23.89,149.0,-0.49,0.97,17.49,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.22,41.0,H-MC,6.85,101118.0,-30717.0,72936.0,-1.58,-23.30,72.13,32.03,98.0,-0.42,0.72,13.26,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,42.74,48.0,M-MC,12.59,225806.0,844.0,166803.0,-0.79,0.38,73.87,74.53,192.0,0.01,1.62,32.75,XY24,BTT,STEEL
32,HINDZINC,730.22,32.35,63.0,M-LC,8.20,215630.0,10554.0,102015.0,-0.52,5.15,47.31,54.89,52.0,0.10,1.54,30.48,X5K,ATH,METALS
79,UNIONBANK,163.00,-13.18,55.0,M-LC,9.34,157543.0,16703.0,27791.0,-0.51,11.86,17.64,31.59,66.0,0.60,1.13,41.49,XY24,NTT,BANKS
86,WIPRO,420.00,-13.79,48.0,M-LC,5.84,151720.0,775.0,108677.0,-1.60,0.51,71.63,72.51,53.0,0.01,1.09,6.57,XR,NTT,IT
11,BANKINDIA,190.00,-25.47,64.0,H-MC,12.30,193573.0,13765.0,98645.0,-0.72,7.66,50.96,62.52,88.0,0.14,1.38,44.35,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,63.0,H-MC,14.51,118402.0,17898.0,11213.0,-0.40,17.81,9.47,28.96,79.0,1.60,0.85,57.82,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,64.0,H-MC,12.30,193573.0,13765.0,98645.0,-0.72,7.66,50.96,62.52,88.0,0.14,1.38,44.35,XR,NTT,BANKS
32,HINDZINC,730.22,32.35,63.0,M-LC,8.20,215630.0,10554.0,102015.0,-0.52,5.15,47.31,54.89,52.0,0.10,1.54,30.48,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.52,127682.0,4346.0,127286.0,0.74,3.52,99.69,106.72,119.0,0.03,0.91,27.73,AR,ATH,MISC
86,WIPRO,420.00,-13.79,48.0,M-LC,5.84,151720.0,775.0,108677.0,-1.60,0.51,71.63,72.51,53.0,0.01,1.09,6.57,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DABUR,735.00,-13.38,32.0,X-MC,3.55,234449.0,-13619.0,122031.0,-1.29,-5.49,52.05,43.70,102.0,-0.11,1.68,8.16,XY24,BTT,FMCG
29,HAVELLS,2069.16,-7.45,33.0,X-MC,0.91,229351.0,-18380.0,93438.0,-1.72,-7.42,40.74,30.30,92.0,-0.20,1.64,4.89,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178760.0,-21746.0,118893.0,-0.91,-10.85,66.51,48.45,90.0,-0.18,1.28,3.08,X40N,ATH,FINANCE
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53162.0,-11844.0,162771.0,-1.02,-18.22,306.18,232.18,198.0,-0.07,0.38,2.00,XY24,NTT,MISC
65,SIEMENS,4671.50,-5.61,36.0,H-LC,1.24,151595.0,-34500.0,81983.0,-3.67,-18.54,54.08,25.51,15.0,-0.42,1.08,11.71,AR,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-7.45,33.0,X-MC,0.91,229351.0,-18380.0,93438.0,-1.72,-7.42,40.74,30.30,92.0,-0.20,1.64,4.89,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-1.50,50.0,X-MC,1.04,206535.0,14793.0,22967.0,-2.85,7.72,11.12,19.69,99.0,0.64,1.48,15.04,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.82,40.0,X-LC,1.21,243192.0,-33331.0,143410.0,-1.33,-12.05,58.97,39.81,54.0,-0.23,1.74,16.55,XY24,NTT,AUTO
65,SIEMENS,4671.50,-5.61,36.0,H-LC,1.24,151595.0,-34500.0,81983.0,-3.67,-18.54,54.08,25.51,15.0,-0.42,1.08,11.71,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.81,38.0,X-MC,1.66,136042.0,-91.0,31222.0,-1.31,-0.07,22.95,22.86,91.0,-0.00,0.97,12.54,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.55,38.0,X-LC,10.47,282602.0,-33240.0,145907.0,-0.16,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53162.0,-11844.0,162771.0,-1.02,-18.22,306.18,232.18,198.0,-0.07,0.38,2.00,XY24,NTT,MISC
22,COLPAL,3767.14,-5.42,37.0,X-MC,7.47,220850.0,-42515.0,155854.0,-0.87,-16.14,70.57,43.04,84.0,-0.27,1.58,2.53,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178760.0,-21746.0,118893.0,-0.91,-10.85,66.51,48.45,90.0,-0.18,1.28,3.08,X40N,ATH,FINANCE
42,ITC,452.00,-39.61,42.0,X-LC,2.55,195502.0,-4636.0,25533.0,-0.74,-2.32,13.06,10.44,4.0,-0.18,1.40,3.12,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53162.0,-11844.0,162771.0,-1.02,-18.22,306.18,232.18,198.0,-0.07,0.38,2.00,XY24,NTT,MISC
56,RELAXO,1176.00,-40.36,37.0,X-SC,2.87,81301.0,-63859.0,139789.0,-0.49,-43.99,171.94,52.31,136.0,-0.46,0.58,8.83,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.44,41.0,X-SC,4.39,96570.0,-32100.0,77401.0,-0.47,-24.95,80.15,35.20,219.0,-0.41,0.69,10.84,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-5.38,49.0,X-SC,1.21,107660.0,5656.0,38273.0,-0.41,5.54,35.55,43.06,122.0,0.15,0.77,23.61,X40N,ATH,MISC
33,HONAUT,58357.33,-22.75,47.0,X-SC,2.19,108915.0,-19023.0,66155.0,-0.47,-14.87,60.74,36.84,143.0,-0.29,0.78,11.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,51.0,X-LC,12.16,284660.0,-61296.0,129549.0,-1.09,-17.72,45.51,19.73,1.0,-0.47,2.04,4.84,X40,ATH,IT
40,INFY,2275.00,-18.52,51.0,X-LC,7.64,317008.0,4120.0,167570.0,-1.76,1.32,52.86,54.87,3.0,0.02,2.27,8.05,X40,BTT,IT
42,ITC,452.00,-39.61,42.0,X-LC,2.55,195502.0,-4636.0,25533.0,-0.74,-2.32,13.06,10.44,4.0,-0.18,1.40,3.12,X40,NTT,FMCG
83,VBL,671.64,-21.55,38.0,X-LC,10.47,282602.0,-33240.0,145907.0,-0.16,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,42.0,H-LC,7.44,246240.0,-15379.0,134595.0,-1.08,-5.88,54.66,45.57,7.0,-0.11,1.76,4.78,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,53.0,L-SC,29.14,83078.0,-30471.0,70525.0,-1.90,-26.83,84.89,35.27,268.0,-0.43,0.59,103.34,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,41.0,H-SC,10.01,90030.0,-7950.0,29557.0,-1.54,-8.11,32.83,22.05,152.0,-0.27,0.64,30.87,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,63.0,H-MC,14.51,118402.0,17898.0,11213.0,-0.40,17.81,9.47,28.96,79.0,1.60,0.85,57.82,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,62.0,H-SC,11.31,131193.0,-11286.0,30804.0,-0.39,-7.92,23.48,13.70,163.0,-0.37,0.94,54.84,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.35,144900.0,-4320.0,148494.0,-0.75,-2.90,102.48,96.62,208.0,-0.03,1.04,56.81,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.35,144900.0,-4320.0,148494.0,-0.75,-2.90,102.48,96.62,208.0,-0.03,1.04,56.81,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,61.0,H-SC,9.87,150736.0,-28335.0,94059.0,-2.14,-15.82,62.40,36.71,135.0,-0.30,1.08,49.56,XR,NTT,BANKS
64,SHALBY,327.00,1185.49,57.0,M-SC,22.37,174775.0,-6368.0,51838.0,0.06,-3.52,29.66,25.10,235.0,-0.12,1.25,40.06,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,32.35,63.0,M-LC,8.20,215630.0,10554.0,102015.0,-0.52,5.15,47.31,54.89,52.0,0.10,1.54,30.48,X5K,ATH,METALS
68,STARHEALTH,761.00,16.12,65.0,H-SC,7.81,258754.0,-22369.0,155227.0,-0.65,-7.96,59.99,47.26,144.0,-0.14,1.85,38.63,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.18
1,25,43.53
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.99
LC    32.31
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      14.64
X40N     11.27
XR       10.94
XY25     10.42
AR        8.10
OX40N     7.58
X200      1.75
MH        1.64
X5K       1.54
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.47
X-LC    22.95
X-MC    17.18
M-SC    12.54
H-MC     4.60
H-LC     4.59
X-SC     4.53
M-LC     3.76
M-MC     1.62
L-SC     1.45
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.81,-6.34,41.38
IT,12.83,-17.25,80.75
FINANCE,10.47,-15.44,71.19
MISC,6.91,-18.74,84.54
BANKS,6.45,-10.71,68.95
PAINTS,5.52,-19.71,37.55
ELECTRICAL,5.26,-11.86,51.10
INSURANCE,3.79,-4.30,43.29
AC,3.53,1.79,14.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240163.0,22
XR,1378990.0,14
AR,1195622.0,9
X40,793935.0,10
X40N,660266.0,9
OX40N,585961.0,10
XY25,481422.0,7
SR,266044.0,2
X5K,102015.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3440804.0,25
M-SC,1286622.0,15
X-MC,1134127.0,12
X-LC,1032532.0,13
X-SC,547800.0,6
H-MC,365834.0,4
H-LC,297949.0,3
L-SC,257956.0,3
M-LC,238483.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1200893.0      6
           AR         813610.0      5
           XR         811956.0      7
M-SC       XY24       691557.0      6
X-MC       XY24       593802.0      4
X-LC       X40        492125.0      5
X-SC       X40N       307628.0      4
H-SC       OX40N      274489.0      4
           SR         266044.0      2
X-MC       X40        224409.0      4
H-LC       AR         216578.0      2
X-LC       X40N       215543.0      3
           XY24       213506.0      2
H-MC       XY24       183040.0      1
X-MC       XY25       178821.0      2
L-SC       XR         168816.0      2
M-MC       XY24       166803.0      1
M-SC       AR         165434.0      2
X-SC       XY24       162771.0      1
M-SC       OX40N      149396.0      4
           XY25       148494.0      1
X-MC       X40N       137095.0      2
M-SC       XR         131741.0      2
X-LC       XY25       111358.0      3
M-LC       XR         108677.0      1
           X5K        102015.0      1
H-MC       XR          98645.0      1
L-SC       OX40N       89140.0      1
H-LC       X200        81371.0      1
X-SC       X40         77401.0      1
H-SC       MH          73812.0      1
H-MC       OX40N       72936.0      1
L-MC       XR          59155.0      1
L-LC       XY25        42749.0      1
M-LC       XY24        27791.0      1
H-MC       MH          11213.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
